In [1]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [2]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2021-03-07 15:16:02--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.20.189
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.20.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  --.-KB/s    in 0.02s   

2021-03-07 15:16:02 (27.8 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [3]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [4]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [5]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [6]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [7]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [8]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [9]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]), tensor(23))

In [10]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions

In [29]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [30]:
model(X[0:1])

tensor([[[-1.7783e-02,  2.4125e-02, -1.0223e-01,  2.2989e-02, -3.0662e-02,
           4.5386e-02, -1.9198e-01,  5.2129e-02, -2.4974e-05,  9.2945e-03,
          -2.8091e-03,  3.5611e-02, -1.7778e-02,  2.5900e-02, -3.4008e-03,
           1.3593e-01,  4.4603e-02, -2.1362e-02, -5.2471e-02, -5.7426e-03,
          -1.3712e-01,  7.6828e-02,  3.0541e-02, -6.4586e-02, -6.1779e-02,
          -2.2246e-02,  2.7849e-02]]], grad_fn=<AddBackward0>)

In [34]:
model = model.cuda()

In [31]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [23]:
generate_text()

ve that love can do everything it is the|bzkoyijndgtvejnxiwuuwmxvyfr eamgmxuaddpd


In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [38]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        answers = answers.squeeze(0)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



Epoch 0. Time: 4.357, Train loss: 2.176
nating feelings a waking sleep an enduri|pdjditsaohcfrdbguaupgaqrabyyccbe dfcihcp



Epoch 1. Time: 4.451, Train loss: 1.785
ter and inscribes progress of woman on h|yb bhlhvwngpbikej lqwznznym nnmscfrqexf 



Epoch 2. Time: 4.222, Train loss: 1.641
ion the consequence is the jews a people|slzbpmtcwqrez vfrybhllgndib wwmkppvyxvah



Epoch 3. Time: 4.297, Train loss: 1.556
uced to imperfect men to slaves and inst|icjpbfcfqdvaxxstpkihujavmhknxebgjctfywsn


...


Epoch 95. Time: 4.364, Train loss: 0.935
entical with them perhaps but who wishes|ssueqw zfsgevzx  vuhze wzzfjvxcvd qqctpd



Epoch 96. Time: 4.516, Train loss: 0.937
 that is the unmistakable declaration an|kxlpwfftfnjzgr n fca oj yuckesiizdfydezu



Epoch 97. Time: 4.662, Train loss: 0.934
t easier on a given occasion to produce |dclxkvwerb lczsjmkhultinfdeexdrskjngmdil



Epoch 98. Time: 4.413, Train loss: 0.930
 they had given comfort to the sufferers|l qjlqabvcxqhqngcbyesnammgiwthpaeiakcdbj



Epoch 99. Time: 4.657, Train loss: 0.927
itself repeatedly but just here is where|mzjkafknialjlsqrgkqbeww juprgyihfgreuwwp
